<a href="https://colab.research.google.com/github/girinath18/AI_Headshot_RG/blob/headshots/Nokia2API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install pyngrok

In [34]:
from pyngrok import ngrok

In [35]:
ngrok.set_auth_token("2grhewRoUtjHE4SVcxmbIPobZ9e_7KdSgCrcQoczPKpuwUtpm")
public_url = ngrok.connect(5000).public_url

In [11]:
!git clone https://github.com/girinath18/roop.git

Cloning into 'roop'...
remote: Enumerating objects: 1540, done.
remote: Total 1540 (delta 0), reused 0 (delta 0), pack-reused 1540
Receiving objects: 100% (1540/1540), 97.43 MiB | 47.31 MiB/s, done.
Resolving deltas: 100% (917/917), done.


In [36]:
!pip install flask_ngrok
!pip install onnxruntime
!pip install customtkinter
!pip install torch torchvision
!pip install flask
!pip install pyngrok --upgrade
!pip install --upgrade pyngrok
!pip install -r /content/roop/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring tkinterdnd2-universal: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring onnxruntime: markers 'python_version != "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-coreml: markers 'python_version == "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment


# **To changing the directory**

In [37]:
%cd roop

[Errno 2] No such file or directory: 'roop'
/content/roop/roop


# **Basic logic code but working code API script**

In [ ]:
from flask import Flask, request, jsonify
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)


TEMPLATES_FOLDER_PATH = "/content/drive/MyDrive/template/captain/Female"

def face_swap(source_image_folder, output_base_folder):
    try:

        output_folder = os.path.join(output_base_folder, "(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")


        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")


        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")


        for source_image_name in source_images:
            source_image_path = os.path.join(source_image_folder, source_image_name)


            if os.path.isfile(source_image_path):
                logging.info(f"Processing source image: {source_image_name}")

                processed_image_paths = set()

                for target_image_name in template_images:
                    target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)


                    if os.path.isfile(target_image_path):
                        logging.info(f"Processing target image: {target_image_name}")

                        output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                        if output_image_path in processed_image_paths:
                            logging.info(f"Skipping already processed image: {output_image_path}")
                            continue


                        command = (
                            f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                            f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                            f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                            f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                        )
                        logging.info(f"Running command: {command}")

                        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                        stdout, stderr = process.communicate()
                        if process.returncode != 0:
                            logging.error(f"Failed to process target image: {target_image_name}")
                            logging.error(f"Command output: {stderr.decode('utf-8')}")
                        else:
                            logging.info(f"Successfully processed target image: {target_image_name}")
                            logging.info(f"Command output: {stdout.decode('utf-8')}")
                            processed_image_paths.add(output_image_path)


                        if len(processed_image_paths) >= len(template_images):
                            break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:

        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)


        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        output_base_folder = "/content"

        face_swap(source_image_folder, output_base_folder)


        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':

    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


 * Ngrok tunnel URL: NgrokTunnel: "https://e12b-35-240-203-31.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Failed to process target image: Navy Cap (3).png
Downloading: 529MB [00:02, 190MB/s]                           
Downloading: 332MB [00:04, 70.0MB/s]                          
100%|██████████| 281857/281857 [00:03<00:00, 80275.17KB/s]
Downloading...
From: https://github.com/bhky/opennsfw2/releases/download/v0.1.0/open_nsfw_weights.h5
To: /root/.opennsfw2/weights/open_nsfw_weights.h5
100%|██████████| 24.2M/24.2M [00:00<00:00, 220MB/s]
100%|██████████| 104M/104M [00:00<00:00, 312MB/s]
100%|██████████| 81.4M/81.4M [00:00<00:00, 267MB/s]
Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Navy Cap (1).png
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Navy Cap (26).png
ERROR:root:Command output: Seg

# **Connecting notebook to the Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **New JSON file for sample templates**

In [7]:
{
  "categories": {
    "Mall": {
      "Male": [
        {
          "id": "1",
          "url": "https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2"
        },
        {
          "id": "2",
          "url": "https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2"
        },
        {
          "id": "3",
          "url": "https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2"
        },
        {
          "id": "4",
          "url": "https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2"
        },
        {
          "id": "5",
          "url": "https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2"
        },
        {
          "id": "6",
          "url": "https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2"
        },
        {
          "id": "7",
          "url": "https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2"
        }
      ],
      "Female": [
        {
          "id": "8",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "9",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "10",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "11",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "12",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "13",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "14",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "15",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "16",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "17",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "18",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "19",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        },
        {
          "id": "20",
          "url": "https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP"
        }
      ]
    },
    "Nayagara": {
      "Male": [
        {
          "id": "21",
          "url": "https://drive.google.com/drive/folders/1-TYdIsNt_Hu-1cvIuk07QEBwM3JNr8H-"
        },
        {
          "id": "22",
          "url": "https://drive.google.com/drive/folders/1-TYdIsNt_Hu-1cvIuk07QEBwM3JNr8H-"
        },
        {
          "id": "23",
          "url": "https://drive.google.com/drive/folders/1-TYdIsNt_Hu-1cvIuk07QEBwM3JNr8H-"
        },
        {
          "id": "24",
          "url": "https://drive.google.com/drive/folders/1-TYdIsNt_Hu-1cvIuk07QEBwM3JNr8H-"
        },
        {
          "id": "25",
          "url": "https://drive.google.com/drive/folders/1-TYdIsNt_Hu-1cvIuk07QEBwM3JNr8H-"
        },
        {
          "id": "26",
          "url": "https://drive.google.com/drive/folders/1-TYdIsNt_Hu-1cvIuk07QEBwM3JNr8H-"
        },
        {
          "id": "27",
          "url": "https://drive.google.com/drive/folders/1-TYdIsNt_Hu-1cvIuk07QEBwM3JNr8H-"
        },
        {
          "id": "28",
          "url": "https://drive.google.com/drive/folders/1-TYdIsNt_Hu-1cvIuk07QEBwM3JNr8H-"
        }
      ],
      "Female": [
        {
          "id": "29",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "30",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "31",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "32",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "33",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "34",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "35",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "36",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "37",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        },
        {
          "id": "38",
          "url": "https://drive.google.com/drive/folders/1-bdRq16dLMxSOmqdlpXLvyGp6oZ7hBMX"
        }
      ]
    }
  }
}


{'categories': {'Mall': {'Male': [{'id': '1',
     'url': 'https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2'},
    {'id': '2',
     'url': 'https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2'},
    {'id': '3',
     'url': 'https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2'},
    {'id': '4',
     'url': 'https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2'},
    {'id': '5',
     'url': 'https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2'},
    {'id': '6',
     'url': 'https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2'},
    {'id': '7',
     'url': 'https://drive.google.com/drive/folders/13BBrnYNBRBlybsemOi9WYTnm3zIEWLR2'}],
   'Female': [{'id': '8',
     'url': 'https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP'},
    {'id': '9',
     'url': 'https://drive.google.com/drive/folders/11Lg4gyOPV0om7isdbbnmvs9i5K_uTtGP'},
    {'id': '10

# **Creating the json file for template API**

In [ ]:
{
  "categories": {
    "category1": [
      "https://drive.google.com/uc?id=1-T922LMrBkHBQL9OuM8ZMWcW91k19oot",
      "https://drive.google.com/uc?id=1-NTz8uGFjXHOfweygfbZI3UaZ7sBGvtt",
      "https://drive.google.com/uc?id=1-EVrOQOBrrH4cIndzFjGQr58gyPBe1Sh",
      "https://drive.google.com/uc?id=1-Ch2-ANY3_r72jwQ63macpj-Ju-HsmVl",
      "https://drive.google.com/uc?id=1-Ay6QO8Bt2Lr97F8jscb22DiQ-MRhNe9",
      "https://drive.google.com/uc?id=1-ADxhsyYbYtfjsuFtHOJlUM52FlkKTDi"
    ],
    "category2": [
      "https://drive.google.com/uc?id=10HQHbfjV_33xa7WhZeYS4DMuQBqNpw3s",
      "https://drive.google.com/uc?id=109B1r5auRVDKuUJhzn0TrLHhbQf7depB",
      "https://drive.google.com/uc?id=1-sZ8lV6OGo4p1Kp7PPah58cZB0KfOiJ1",
      "https://drive.google.com/uc?id=1-sJ-qYh9gAYHrbujcdBUaKOadLUuFtjj",
      "https://drive.google.com/uc?id=1-poq_gCGlIWoTKRQFXgCGs1t_IAjPH2l",
      "https://drive.google.com/uc?id=1-lmK7Fq_7UpqP_a1gP969MFgggfebKwI",
      "https://drive.google.com/uc?id=1-gNkqHvt_wHnQyTskO-SIdxhlB_vDH_o",
      "https://drive.google.com/uc?id=1-ePjWcUSawXII4El6iiI27p05dZkUjgm",
      "https://drive.google.com/uc?id=10Zu6mPX9G50Bi89CkzBTgaIIOe9jPQaP",
      "https://drive.google.com/uc?id=10S62IrVbLyIa7rDlm0v6bL4hw88iCay-",
      "https://drive.google.com/uc?id=10RazuWfDgbAjUQ8e7LnQhqjvZEzwNWfA",
      "https://drive.google.com/uc?id=10M_UvI7fhyOSz4JuHyrdRx1r2QePNCki",
      "https://drive.google.com/uc?id=10LGIUYQptd6O91ld2KaCw52LdT9VVgf5"
    ]
  }
}


{'categories': {'category1': ['https://drive.google.com/uc?id=1-T922LMrBkHBQL9OuM8ZMWcW91k19oot',
   'https://drive.google.com/uc?id=1-NTz8uGFjXHOfweygfbZI3UaZ7sBGvtt',
   'https://drive.google.com/uc?id=1-EVrOQOBrrH4cIndzFjGQr58gyPBe1Sh',
   'https://drive.google.com/uc?id=1-Ch2-ANY3_r72jwQ63macpj-Ju-HsmVl',
   'https://drive.google.com/uc?id=1-Ay6QO8Bt2Lr97F8jscb22DiQ-MRhNe9',
   'https://drive.google.com/uc?id=1-ADxhsyYbYtfjsuFtHOJlUM52FlkKTDi'],
  'category2': ['https://drive.google.com/uc?id=10HQHbfjV_33xa7WhZeYS4DMuQBqNpw3s',
   'https://drive.google.com/uc?id=109B1r5auRVDKuUJhzn0TrLHhbQf7depB',
   'https://drive.google.com/uc?id=1-sZ8lV6OGo4p1Kp7PPah58cZB0KfOiJ1',
   'https://drive.google.com/uc?id=1-sJ-qYh9gAYHrbujcdBUaKOadLUuFtjj',
   'https://drive.google.com/uc?id=1-poq_gCGlIWoTKRQFXgCGs1t_IAjPH2l',
   'https://drive.google.com/uc?id=1-lmK7Fq_7UpqP_a1gP969MFgggfebKwI',
   'https://drive.google.com/uc?id=1-gNkqHvt_wHnQyTskO-SIdxhlB_vDH_o',
   'https://drive.google.com/uc?id=1

# **creating your Flask app to serve the JSON data**

In [24]:
!pip install flask flask-ngrok

In [ ]:
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok
import json
import time
import requests

app = Flask(__name__)
run_with_ngrok(app)


json_path = '/content/drive/MyDrive/templates/categories.json'
with open(json_path) as f:
    data = json.load(f)

@app.route('/categories', methods=['GET'])
def get_categories():
    return jsonify(data)

if __name__ == '__main__':
    try:
        app.run()
    except Exception as e:
        print(f"Error occurred: {e}")

        time.sleep(5)
        app.run()


In [ ]:
from flask import Flask, jsonify
import json
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

# Load your JSON data
json_path = '/content/drive/MyDrive/templates/categories.json'
with open(json_path) as f:
    data = json.load(f)

@app.route('/categories', methods=['GET'])
def get_categories():
    return jsonify(data)

@app.route('/categories/<category_id>', methods=['GET'])
def get_category_by_id(category_id):
    category = data['categories'].get(category_id)
    if category:
        return jsonify(category)
    else:
        return jsonify({'error': 'Category not found'}), 404

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c7df-35-230-47-52.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/May/2024 11:30:49] "GET /categories/8 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/May/2024 11:32:04] "GET /categories HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/May/2024 11:33:10] "GET /categories/Nayagara HTTP/1.1" 200 -


# **Added API to display output images**

In [ ]:
from flask import Flask, request, jsonify, send_from_directory
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

TEMPLATES_FOLDER_PATH = "/content/drive/MyDrive/template/doctor/Female"
OUTPUT_BASE_FOLDER = "/content/drive/MyDrive"

def face_swap(source_image_folder, output_base_folder):
    try:
        output_folder = os.path.join(output_base_folder, "(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")


        source_image_name = source_images[0]
        source_image_path = os.path.join(source_image_folder, source_image_name)

        if os.path.isfile(source_image_path):
            logging.info(f"Processing source image: {source_image_name}")

            processed_image_paths = set()

            for target_image_name in template_images:
                target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                if os.path.isfile(target_image_path):
                    logging.info(f"Processing target image: {target_image_name}")

                    output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                    if output_image_path in processed_image_paths:
                        logging.info(f"Skipping already processed image: {output_image_path}")
                        continue

                    command = (
                        f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                        f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                        f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                        f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                    )
                    logging.info(f"Running command: {command}")

                    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    stdout, stderr = process.communicate()
                    if process.returncode != 0:
                        logging.error(f"Failed to process target image: {target_image_name}")
                        logging.error(f"Command output: {stderr.decode('utf-8')}")
                    else:
                        logging.info(f"Successfully processed target image: {target_image_name}")
                        logging.info(f"Command output: {stdout.decode('utf-8')}")
                        processed_image_paths.add(output_image_path)

                    if len(processed_image_paths) >= len(template_images):
                        break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)

        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        face_swap(source_image_folder, OUTPUT_BASE_FOLDER)

        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images', methods=['GET'])
def output_images_api():
    try:
        output_folder = os.path.join(OUTPUT_BASE_FOLDER, "(Outputs)")
        if not os.path.exists(output_folder):
            logging.error("Output folder does not exist.")
            return jsonify({"error": "Output folder does not exist"}), 404

        output_images = os.listdir(output_folder)
        if not output_images:
            logging.info("No output images found.")
            return jsonify({"message": "No output images found."}), 200

        return jsonify({"output_images": output_images}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images/<filename>', methods=['GET'])
def get_output_image(filename):
    try:
        output_folder = os.path.join(OUTPUT_BASE_FOLDER, "(Outputs)")
        return send_from_directory(output_folder, filename)
    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


 * Ngrok tunnel URL: NgrokTunnel: "https://61b8-35-240-203-31.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Failed to process target image: Doctor (1).png
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Doctor (2).png
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Doctor (5).png
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Doctor (3).png
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Doctor (4).png
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: Doctor (6).png
ERROR:root:Command output: Segmentation fault (core dumped)

INFO:werkzeug:127.0.0.1 - - [23/May/2024 11:22:14] "POST /faceswap HTTP/1.1" 200 -

# **changes the output to be displayed in Zipfile**

In [ ]:
from flask import Flask, request, jsonify, send_from_directory, send_file
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess
import zipfile
import io

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

TEMPLATES_FOLDER_PATH = "/content/drive/MyDrive/template/doctor/Female"
OUTPUT_BASE_FOLDER = "/content/drive/MyDrive"

def face_swap(source_image_folder, output_base_folder):
    try:
        output_folder = os.path.join(output_base_folder, "(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")

        source_image_name = source_images[0]
        source_image_path = os.path.join(source_image_folder, source_image_name)

        if os.path.isfile(source_image_path):
            logging.info(f"Processing source image: {source_image_name}")

            processed_image_paths = set()

            for target_image_name in template_images:
                target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                if os.path.isfile(target_image_path):
                    logging.info(f"Processing target image: {target_image_name}")

                    output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                    if output_image_path in processed_image_paths:
                        logging.info(f"Skipping already processed image: {output_image_path}")
                        continue

                    command = (
                        f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                        f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                        f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                        f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                    )
                    logging.info(f"Running command: {command}")

                    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    stdout, stderr = process.communicate()
                    if process.returncode != 0:
                        logging.error(f"Failed to process target image: {target_image_name}")
                        logging.error(f"Command output: {stderr.decode('utf-8')}")
                    else:
                        logging.info(f"Successfully processed target image: {target_image_name}")
                        logging.info(f"Command output: {stdout.decode('utf-8')}")
                        processed_image_paths.add(output_image_path)

                    if len(processed_image_paths) >= len(template_images):
                        break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)

        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        face_swap(source_image_folder, OUTPUT_BASE_FOLDER)

        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images', methods=['GET'])
def output_images_api():
    try:
        output_folder = os.path.join(OUTPUT_BASE_FOLDER, "(Outputs)")
        if not os.path.exists(output_folder):
            logging.error("Output folder does not exist.")
            return jsonify({"error": "Output folder does not exist"}), 404

        output_images = os.listdir(output_folder)
        if not output_images:
            logging.info("No output images found.")
            return jsonify({"message": "No output images found."}), 200

        # Create a zip file in memory
        memory_file = io.BytesIO()
        with zipfile.ZipFile(memory_file, 'w') as zf:
            for filename in output_images:
                filepath = os.path.join(output_folder, filename)
                zf.write(filepath, filename)
        memory_file.seek(0)

        return send_file(memory_file, attachment_filename='output_images.zip', as_attachment=True)

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images/<filename>', methods=['GET'])
def get_output_image(filename):
    try:
        output_folder = os.path.join(OUTPUT_BASE_FOLDER, "(Outputs)")
        return send_from_directory(output_folder, filename)
    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


# **This code working as expected**

In [ ]:
from flask import Flask, request, jsonify, send_from_directory, send_file
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess
import zipfile
import io

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

TEMPLATES_FOLDER_PATH = "/content/drive/MyDrive/templates/kings"
OUTPUT_BASE_FOLDER = "/content/drive/MyDrive"

def face_swap(source_image_folder, output_base_folder):
    try:
        output_folder = os.path.join(output_base_folder, "(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")

        source_image_name = source_images[0]
        source_image_path = os.path.join(source_image_folder, source_image_name)

        if os.path.isfile(source_image_path):
            logging.info(f"Processing source image: {source_image_name}")

            processed_image_paths = set()

            for target_image_name in template_images:
                target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                if os.path.isfile(target_image_path):
                    logging.info(f"Processing target image: {target_image_name}")

                    output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                    if output_image_path in processed_image_paths:
                        logging.info(f"Skipping already processed image: {output_image_path}")
                        continue

                    command = (
                        f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                        f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                        f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                        f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                    )
                    logging.info(f"Running command: {command}")

                    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    stdout, stderr = process.communicate()
                    if process.returncode != 0:
                        logging.error(f"Failed to process target image: {target_image_name}")
                        logging.error(f"Command output: {stderr.decode('utf-8')}")
                    else:
                        logging.info(f"Successfully processed target image: {target_image_name}")
                        logging.info(f"Command output: {stdout.decode('utf-8')}")
                        processed_image_paths.add(output_image_path)

                    if len(processed_image_paths) >= len(template_images):
                        break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        source_image_folder = "/content/source_images"
        os.makedirs(source_image_folder, exist_ok=True)

        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        face_swap(source_image_folder, OUTPUT_BASE_FOLDER)

        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated."}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images', methods=['GET'])
def output_images_api():
    try:
        output_folder = os.path.join(OUTPUT_BASE_FOLDER, "RG(Outputs)")
        if not os.path.exists(output_folder):
            logging.error("Output folder does not exist.")
            return jsonify({"error": "Output folder does not exist"}), 404

        output_images = os.listdir(output_folder)
        if not output_images:
            logging.info("No output images found.")
            return jsonify({"message": "No output images found."}), 200


        memory_file = io.BytesIO()
        with zipfile.ZipFile(memory_file, 'w') as zf:
            for filename in output_images:
                filepath = os.path.join(output_folder, filename)
                zf.write(filepath, filename)
        memory_file.seek(0)

        return send_file(memory_file, download_name='output_images.zip', as_attachment=True)

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images/<filename>', methods=['GET'])
def get_output_image(filename):
    try:
        output_folder = os.path.join(OUTPUT_BASE_FOLDER, "(Outputs)")
        return send_from_directory(output_folder, filename)
    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


 * Ngrok tunnel URL: NgrokTunnel: "https://9136-34-90-35-134.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Failed to process target image: KNG (10).jpg
Downloading: 529MB [00:06, 87.0MB/s]                          
Downloading: 332MB [00:02, 146MB/s]                           
100%|██████████| 281857/281857 [00:03<00:00, 71431.09KB/s]
Downloading...
From: https://github.com/bhky/opennsfw2/releases/download/v0.1.0/open_nsfw_weights.h5
To: /root/.opennsfw2/weights/open_nsfw_weights.h5
100%|██████████| 24.2M/24.2M [00:00<00:00, 73.7MB/s]
100%|██████████| 104M/104M [00:00<00:00, 206MB/s] 
100%|██████████| 81.4M/81.4M [00:00<00:00, 233MB/s]
Segmentation fault (core dumped)

ERROR:root:Failed to process target image: KNG (9).jpg
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: KNG (12).jpg
ERROR:root:Command output: Segmentation fa

# **Trying to store the output images file as temprary file**

In [ ]:
from flask import Flask, request, jsonify, send_from_directory, send_file
import os
import shutil
import logging
from pyngrok import ngrok
import subprocess
import zipfile
import io
import tempfile

app = Flask(__name__)

logging.basicConfig(level=logging.INFO)

TEMPLATES_FOLDER_PATH = "/content/drive/MyDrive/template/kings"

def face_swap(source_image_folder, temp_output_folder):
    try:
        output_folder = os.path.join(temp_output_folder, "(Outputs)")
        os.makedirs(output_folder, exist_ok=True)
        logging.info(f"Output folder created at: {output_folder}")

        source_images = os.listdir(source_image_folder)
        if not source_images:
            logging.error("No source images found.")
            return
        logging.info(f"Found {len(source_images)} source images.")

        template_images = os.listdir(TEMPLATES_FOLDER_PATH)
        if not template_images:
            logging.error("No template images found.")
            return
        logging.info(f"Found {len(template_images)} template images.")

        source_image_name = source_images[0]
        source_image_path = os.path.join(source_image_folder, source_image_name)

        if os.path.isfile(source_image_path):
            logging.info(f"Processing source image: {source_image_name}")

            processed_image_paths = set()

            for target_image_name in template_images:
                target_image_path = os.path.join(TEMPLATES_FOLDER_PATH, target_image_name)

                if os.path.isfile(target_image_path):
                    logging.info(f"Processing target image: {target_image_name}")

                    output_image_path = os.path.join(output_folder, f"{source_image_name}_{target_image_name}")
                    if output_image_path in processed_image_paths:
                        logging.info(f"Skipping already processed image: {output_image_path}")
                        continue

                    command = (
                        f'python run.py -s "{source_image_path}" -t "{target_image_path}" '
                        f'-o "{output_image_path}" --keep-fps --execution-threads 14 --many-faces '
                        f'--execution-provider cuda --frame-processor face_swapper face_enhancer '
                        f'--output-video-quality 35 --temp-frame-format jpg --max-memory 46'
                    )
                    logging.info(f"Running command: {command}")

                    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    stdout, stderr = process.communicate()
                    if process.returncode != 0:
                        logging.error(f"Failed to process target image: {target_image_name}")
                        logging.error(f"Command output: {stderr.decode('utf-8')}")
                    else:
                        logging.info(f"Successfully processed target image: {target_image_name}")
                        logging.info(f"Command output: {stdout.decode('utf-8')}")
                        processed_image_paths.add(output_image_path)

                    if len(processed_image_paths) >= len(template_images):
                        break

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

@app.route('/faceswap', methods=['POST'])
def faceswap_api():
    try:
        source_image_folder = tempfile.mkdtemp()
        logging.info(f"Temporary source image folder created at: {source_image_folder}")

        source_images = request.files.getlist('source_images')
        if not source_images:
            logging.error("No images uploaded.")
            return jsonify({"error": "No images uploaded"}), 400

        for source_image in source_images:
            source_image.save(os.path.join(source_image_folder, source_image.filename))
            logging.info(f"Uploaded image: {source_image.filename}")

        temp_output_folder = tempfile.mkdtemp()
        logging.info(f"Temporary output folder created at: {temp_output_folder}")

        face_swap(source_image_folder, temp_output_folder)

        shutil.rmtree(source_image_folder)

        return jsonify({"message": "Face swap process initiated.", "output_folder": temp_output_folder}), 200

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images', methods=['GET'])
def output_images_api():
    try:
        temp_output_folder = request.args.get('temp_output_folder')
        if not temp_output_folder or not os.path.exists(temp_output_folder):
            logging.error("Output folder does not exist.")
            return jsonify({"error": "Output folder does not exist"}), 404

        output_folder = os.path.join(temp_output_folder, "(Outputs)")
        output_images = os.listdir(output_folder)
        if not output_images:
            logging.info("No output images found.")
            return jsonify({"message": "No output images found."}), 200

        # Create a zip file in memory
        memory_file = io.BytesIO()
        with zipfile.ZipFile(memory_file, 'w') as zf:
            for filename in output_images:
                filepath = os.path.join(output_folder, filename)
                zf.write(filepath, filename)
        memory_file.seek(0)

        shutil.rmtree(temp_output_folder)  # Clean up the temporary output folder

        return send_file(memory_file, download_name='output_images.zip', as_attachment=True)

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

@app.route('/output_images/<filename>', methods=['GET'])
def get_output_image(filename):
    try:
        temp_output_folder = request.args.get('temp_output_folder')
        if not temp_output_folder or not os.path.exists(temp_output_folder):
            logging.error("Output folder does not exist.")
            return jsonify({"error": "Output folder does not exist"}), 404

        output_folder = os.path.join(temp_output_folder, "(Outputs)")
        return send_from_directory(output_folder, filename)

    except Exception as e:
        logging.exception(f"An error occurred: {e}")
        return jsonify({"error": "An error occurred"}), 500

if __name__ == '__main__':
    url = ngrok.connect(5000)
    print(f" * Ngrok tunnel URL: {url}")

    app.run(port=5000)


 * Ngrok tunnel URL: NgrokTunnel: "https://d32b-35-221-146-124.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Failed to process target image: KNG (9).jpg
Downloading: 529MB [00:13, 42.0MB/s]                           
Downloading: 332MB [00:06, 50.5MB/s]                          
100%|██████████| 281857/281857 [00:03<00:00, 70811.66KB/s]
Downloading...
From: https://github.com/bhky/opennsfw2/releases/download/v0.1.0/open_nsfw_weights.h5
To: /root/.opennsfw2/weights/open_nsfw_weights.h5
100%|██████████| 24.2M/24.2M [00:00<00:00, 54.9MB/s]
100%|██████████| 104M/104M [00:01<00:00, 85.2MB/s]
100%|██████████| 81.4M/81.4M [00:00<00:00, 92.4MB/s]
Segmentation fault (core dumped)

ERROR:root:Failed to process target image: KNG (10).jpg
ERROR:root:Command output: Segmentation fault (core dumped)

ERROR:root:Failed to process target image: KNG (12).jpg
ERROR:root:Command output: Segmentation 